In [1]:
import os
import glob
import cv2
import numpy as np

imagesPath = "Images\\"

def nothing(x):
    pass

def main():
    for filename in glob.glob(imagesPath+"*.jpg"):
        img = cv2.imread(filename)
        print(filename, img.shape)

        #############################
        scale = 0.5
        widthNew = int(img.shape[1] * scale)
        heightNew = int(img.shape[0] * scale)
        dim = (widthNew, heightNew)
        img = cv2.resize(img, dim)

        ###########################
        imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        cv2.imshow("gray_"+filename, imgGray)

        ##########################
        imgBlur = cv2.GaussianBlur(imgGray, (15,15), 3)
        cv2.imshow("blur"+filename, imgBlur)

        ##########################
        #imgCanny = cv2.Canny(imgBlur, 22, 50)
        #cv2.imshow("canny"+filename, imgCanny)
        
        ##########################
        imgThresh = cv2.adaptiveThreshold(imgBlur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,35,1.5)
        cv2.imshow("thresh_"+filename, imgThresh)

        ##########################
        imgContours = img.copy()
        contours, hierarchy = cv2.findContours(imgThresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        #print(contours)
        cv2.drawContours(imgContours, contours, -1, (0,0,255), 2)
        cv2.imshow("contours_"+filename, imgContours)
        cv2.waitKey(0)

        #########################
        rectCon = []
        for i in contours:
            area = cv2.contourArea(i)
            
            if area > 500:
                print("area", area)
                peri = cv2.arcLength(i, True)
                approx = cv2.approxPolyDP(i, 0.02*peri, True)
                print(len(approx))
                if len(approx) == 4:
                    print(area)
                    #print(len(approx))
                    rectCon.append(i)
        rectCon = sorted(rectCon, key=cv2.contourArea, reverse=True)  

        #########################
        imgContours2 = img.copy()
        biggestContour = rectCon[0]
        #print(rectCon[0])
        peri = cv2.arcLength(biggestContour, True)
        approx = cv2.approxPolyDP(biggestContour, 0.02*peri, True)
        print(approx)
        print(approx[0][0])

        cv2.drawContours(imgContours2, approx, -1, (0,0,255), 8)

        cv2.imshow("contours2_"+filename, imgContours2)

        ##################################################################
        points = findPositionCorner(approx)
        widthKTP = 850
        heightKTP = 540
        print(points)
        pts1 = np.float32(points)
        pts2 = np.float32([[0,0],[widthKTP,0],[0,heightKTP],[widthKTP,heightKTP]])
        print(pts1, pts2)
        matrix = cv2.getPerspectiveTransform(pts1, pts2)
        imgWarpColored = cv2.warpPerspective(img, matrix, (widthKTP, heightKTP))

        


        cv2.imshow("wrap_"+filename, imgWarpColored)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

def findPositionCorner(myPoints):

    myPoints = myPoints.reshape((4, 2))
    myPointsNew = np.zeros((4, 1, 2), dtype=np.int32)
    add = myPoints.sum(1)
 
    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] = myPoints[np.argmax(add)]
    diff = np.diff(myPoints, axis=1)
    myPointsNew[1] =myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
 
    return myPointsNew

if __name__ == '__main__':       
    # Calling main() function 
    main()

Images\identity_card_example.b686f703-1024x664.jpg (664, 1024, 3)
area 121432.5
4
121432.5
[[[ 43  43]]

 [[ 58 304]]

 [[467 288]]

 [[455  28]]]
[43 43]
[[[ 43  43]]

 [[455  28]]

 [[ 58 304]]

 [[467 288]]]
[[[ 43.  43.]]

 [[455.  28.]]

 [[ 58. 304.]]

 [[467. 288.]]] [[  0.   0.]
 [850.   0.]
 [  0. 540.]
 [850. 540.]]
Images\photo6269503147001097552.jpg (960, 1280, 3)
area 231833.0
4
231833.0
[[[ 10  37]]

 [[ 59 479]]

 [[639 419]]

 [[629  70]]]
[10 37]
[[[ 10  37]]

 [[629  70]]

 [[ 59 479]]

 [[639 419]]]
[[[ 10.  37.]]

 [[629.  70.]]

 [[ 59. 479.]]

 [[639. 419.]]] [[  0.   0.]
 [850.   0.]
 [  0. 540.]
 [850. 540.]]
Images\photo6269503147001097554.jpg (1280, 960, 3)
area 976.5
13
area 511.5
8
area 144024.5
4
144024.5
area 601.0
8
area 516.5
6
area 593.0
5
area 2238.5
3
[[[472 227]]

 [[ 33 234]]

 [[ 24 563]]

 [[439 578]]]
[472 227]
[[[ 33 234]]

 [[472 227]]

 [[ 24 563]]

 [[439 578]]]
[[[ 33. 234.]]

 [[472. 227.]]

 [[ 24. 563.]]

 [[439. 578.]]] [[  0.   0.]
 [85

IndexError: list index out of range

*#====CROP and OCR====================*

In [1]:
import glob
import os
import numpy as np
import cv2
import Utils
import easyocr
import json

resultImagesPath = "ResultImages\\"
resultJSONPath = "ResultJSON\\"
NIKPosition = np.array([[190,75],[605,125]])
detailPosition = np.array([[220,125],[605,445]])
fotoPosition = np.array([[610, 105],[815,365]])
ttdPosition = np.array([[610,405],[815,525]])

color = (0,0,255)
thickness = 2
reader = easyocr.Reader(['en'])

def main():
    for filename in glob.glob(resultImagesPath+"*.jpg"):
        img = cv2.imread(filename)
        print(filename, img.shape)

        ###################################
        imgRect = img.copy()
        cv2.rectangle(imgRect, NIKPosition[0], NIKPosition[1], color, thickness)
        cv2.rectangle(imgRect, detailPosition[0], detailPosition[1], color, thickness)
        cv2.rectangle(imgRect, fotoPosition[0], fotoPosition[1], color, thickness)
        cv2.rectangle(imgRect, ttdPosition[0], ttdPosition[1], color, thickness)
        cv2.imshow("rect", imgRect)

        ###################################
        NIKImage = imgRect[NIKPosition[0][1]:NIKPosition[1][1], NIKPosition[0][0]:NIKPosition[1][0]]
        detailImage = imgRect[detailPosition[0][1]:detailPosition[1][1], detailPosition[0][0]:detailPosition[1][0]]
        fotoImage = imgRect[fotoPosition[0][1]:fotoPosition[1][1], fotoPosition[0][0]:fotoPosition[1][0]]
        ttdImage = imgRect[ttdPosition[0][1]:ttdPosition[1][1], ttdPosition[0][0]:ttdPosition[1][0]]
        cv2.imshow("NIKImage", NIKImage)
        cv2.imshow("detailImage", detailImage)
        cv2.imshow("fotoImage", fotoImage)
        cv2.imshow("ttdImage", ttdImage)

        ###################################
        
        NIKResult = reader.readtext(NIKImage)
        detailResult = reader.readtext(detailImage)

        ####################################
        NIK = NIKResult[0][1]
        print("-> NIK:", NIK)

        #print(detailResult)
        Nama = detailResult[0][1]
        TTL = detailResult[1][1]
        jenisKelamin = detailResult[2][1]
        golDarah = detailResult[3][1]
        alamat = detailResult[4][1]
        RTRW = detailResult[5][1]
        KelDesa = detailResult[6][1]
        kecamatan = detailResult[7][1]
        agama = detailResult[8][1]
        statusKawin = detailResult[9][1]
        pekerjaan = detailResult[10][1]
        kewarganegaraan = detailResult[11][1]
        berlakuHingga = detailResult[12][1]

        tempatLahir = TTL[:-11]
        waktuLahir = TTL[-10:]
        tanggalLahir = int(waktuLahir[:2])
        bulanLahir = int(waktuLahir[3:5])
        tahunLahir = int(waktuLahir[-4:])

        if jenisKelamin[:3] == "LAK":
            jenisKelamin = "LAKI-LAKI"
        elif jenisKelamin[:2] == "PE":
            jenisKelamin = "PEREMPUAN"

        if golDarah[-1:] == "0" or golDarah[-1:] == "O":
            golDarah = "O"
        elif golDarah[-1:] == "A":
            golDarah = "A"
        elif golDarah[-2:] == "AB":
            golDarah = "AB"
        elif golDarah[-1:] == "B":
            golDarah = "B"

        RT = RTRW[:3]
        RW = RTRW[-3:]

        if agama[:2] == "IS":
            agama = "ISLAM"
        elif agama[:2] == "BU":
            agama = "BUDDHA"
        elif agama[:2] == "HI":
            agama = "HINDU"
        elif agama[:2] == "KO":
            agama = "KONGHUCU"
        elif agama[8:9] == "P":
            agama = "KRISTEN PROTESTAN"
        elif agama[8:10] == "KA":
            agama = "KRISTEN KATOLIK"
            
        if statusKawin[:2] == "BE":
            statusKawin = "BELUM KAWIN"
        elif statusKawin[:2] == "KA":
            statusKawin = "KAWIN"
        elif statusKawin[-1:] == "P":
            statusKawin = "CERAI HIDUP"
        elif statusKawin[-1:] ==  "I":
            statusKawin = "CERAI MATI"

        if berlakuHingga[:1] == "S":
            berlakuHingga = "SEUMUR HIDUP"

        print("-> Nama:", Nama)
        print("-> Tempat Lahir:", tempatLahir)
        print("-> Tanggal Lahir:", tanggalLahir)
        print("-> Bulan Lahir:", bulanLahir)
        print("-> Tahun Lahir:", tahunLahir)
        print("-> Jenis Kelamin:", jenisKelamin)
        print("-> Golongan Darah:", golDarah)
        print("-> Alamat:", alamat)
        print("-> RT:", RT)
        print("-> RW:", RW)
        print("-> Kel/Desa:", KelDesa)
        print("-> Kecamatan:", kecamatan)
        print("-> Agama:", agama)
        print("-> Status Perkawinan:", statusKawin)
        print("-> Pekerjaan:", pekerjaan)
        print("-> Kewarganegaraan:", kewarganegaraan)
        print("-> Berlaku Hingga:", berlakuHingga)
        print("="*50)

        ktp_dict = {
            "nik" : NIK,
            "nama" : Nama,
            "ttl" : {
                "tempatL" : tempatLahir,
                "tanggalL" : tanggalLahir,
                "bulanL" : bulanLahir,
                "tahunL" : tahunLahir
            },
            "jenisKelamin" : jenisKelamin,
            "golDarah" : golDarah,
            "alamat" : {
                "jalan" : alamat,
                "rt" : RT,
                "rw" : RW,
                "kelDesa" : KelDesa,
                "kecamatan" : kecamatan
            },
            "agama" : agama,
            "statusPerkawinan" : statusKawin,
            "pekerjaan" : pekerjaan,
            "kewarganegaraan" : kewarganegaraan,
            "berlakuHingga" : berlakuHingga
        }
        json_object = json.dumps(ktp_dict, indent = 4)
        # Writing to sample.json
        pathJSONSave = resultJSONPath + NIK + "_" + Nama.split()[0] + ".json"
        with open(pathJSONSave, "w") as outfile:
            outfile.write(json_object)
        
        cv2.waitKey(0)
        cv2.destroyAllWindows()



if __name__ == '__main__':       
    # Calling main() function 
    main()

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [27]:
a = "45-05-1998"
print(a[-4:])
print(a[-7:-5])
print(a[-10:-8])

1998
05
45
